# Modifiação do Dataset e envio para o RDS através do S3

In [ ]:
# Importação das bibliotecas necessárias
import boto3
import pandas as pd
from io import BytesIO
from sqlalchemy import create_engine
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from io import BytesIO
import os

In [ ]:
# Configuração da sessão e cliente S3 da AWS
boto3_session = boto3.Session()
s3_client = boto3_session.client('s3')

# Variáveis de ambiente para conexão com o banco de dados
database_name = os.getenv("DATABASE_NAME")
rds_host = os.getenv("RDS_HOST")
rds_port = os.getenv("RDS_PORT")
rds_user = os.getenv("RDS_USER")
rds_password = os.getenv("RDS_PASSWORD")

# Dowload do Dataset do S3

In [ ]:
# Faz o download do arquivo do S3
bucket_name = os.getenv("BUCKET_NAME")
file_name = 'dataset/Hotel Reservations.csv'
response = s3_client.get_object(Bucket=bucket_name, Key=file_name)
data = response['Body'].read()

    # Carrega os dados do arquivo CSV
hotel_reservations_modified = pd.read_csv(BytesIO(data))


# Modificação do dataset

In [ ]:
# Adiciona a coluna label_avg_price_per_room e remove a coluna avg_price_per_room do dataset
def label_avg_price(price):
    if price <= 85:
        return 1
    elif price < 115:
        return 2
    else:
        return 3

hotel_reservations_modified['label_avg_price_per_room'] = hotel_reservations_modified['avg_price_per_room'].apply(label_avg_price)
hotel_reservations_modified = hotel_reservations_modified.drop('avg_price_per_room', axis=1)

print("Coluna 'label_avg_price_per_room' adicionada e coluna 'avg_price_per_room'.")